In [ ]:
import torch
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torchvision
import os
from torch import nn
from torch.optim import lr_scheduler
from tqdm import tqdm
import torch.optim as optim
import torchvision.transforms as transforms
import PIL
from albumentations import Compose, HorizontalFlip, RandomContrast, Crop, RandomBrightnessContrast, RandomCrop, Flip, RandomSizedCrop, OneOf, PadIfNeeded, Normalize, Resize, RandomCrop

## Определение вычислительного устройства

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Подключения диска с данными

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 37.9 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sat Jan  7 19:03:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    29W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Распаковка данных

In [ ]:
# !unzip /content/drive/MyDrive/nubers/CCPD2019-dl1.zip

## Создание модели

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.in_fea = 0
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 7, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(7),
            nn.ReLU(inplace=True))
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(7, 7, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(7),
            nn.ReLU(inplace=True))
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Sequential(
            nn.Linear(512, 32),
            nn.ReLU(inplace=True))
        self.gru1 = nn.GRU(32, 512)
        self.gru1_b = nn.GRU(32, 512)
        self.gru2 = nn.GRU(512, 512)
        self.gru2_b = nn.GRU(512, 512)
        self.fc2 = nn.Sequential(
            nn.Linear(1024, 66),
            nn.ReLU(inplace=True))
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        self.in_fea = x.size(2) * x.size(3)
        x = torch.reshape(x, (x.size(0), 7, 512))
        x = self.fc1(x)
        x_1, h_1 = self.gru1(x)
        x_2, h_2 = self.gru1_b(x)
        x = x_1 + x_2
        x_1, h_1 = self.gru2(x)
        x_2, h_2 = self.gru2_b(x)
        x = torch.cat((x_1, x_2), 2)
        x = self.fc2(x)
        return x

## Создание класса для формирования кастомного датасета, объявление функции для создания даталоадера

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, work_dir:str, state:bool, transform=None):
        self.work_dir = work_dir
        # self.tta = tta
        self.state = state
        self.transform = transform
        self.policies = transforms.AutoAugmentPolicy.CIFAR10
        self.conv = transforms.Compose([transforms.AutoAugment(self.policies)])
        self.data = self.parsing_data()

    def parsing_data(self):
        list_data = os.listdir(self.work_dir)
        result_dataframe = list()
        for i, image in enumerate(list_data):
            image = image.split('-')[1].split('.')
            result_dataframe.append([f'{self.work_dir}/{list_data[i]}', image[0]])
        return pd.DataFrame(result_dataframe, columns=[0, 1])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx:int):
        if self.state:
            image = cv2.imread(self.data.iloc[idx, 0])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            image = PIL.Image.open(self.data.iloc[idx, 0]).convert('RGB')
        label = self.data.iloc[idx, 1]
        if self.transform:
            if not self.state:
                image = self.conv(image)
                image = np.array(image)
            transforming = self.transform(image=image)
            image = transforming["image"]
        image = torchvision.transforms.functional.to_tensor(image)
        return image, label

def CreateDataloader(data_type:str, transforms, shuffle:bool, state:bool):
    work_dir = f'./CCPD2019-dl1/{data_type}'
    dataset = MyDataset(work_dir, state, transforms)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=shuffle, num_workers=2, pin_memory=True)
    data_size = len(dataset)
    return dataloader, data_size

## Немного некрасивого кода для получения словараей со всеми символами из лэйблов и соответствующими им порядковыми номерами

In [ ]:
work_dir = '/content/CCPD2019-dl1/train'
d = {}
inv_d = {}
list_data = os.listdir(work_dir)
for i, image in enumerate(list_data):
    image = image.split('-')[1].split('.')
    if len(image[0]) != 7:
        print(image[0])
    for sumbul in image[0]:
        if sumbul not in d:
            inv_d[len(d)] = sumbul
            d[sumbul] = len(d)

## Функция конвертирования символьного лэйбла в числовой

In [ ]:
def ConverLabelToNum(d:dict, lbl:torch.Tensor):
    res = list()
    for iteration in lbl:
        tmp = list()
        for sumbul in iteration:
            tmp.append(d[sumbul])
        res.append(tmp)
    return torch.tensor(res)

## Объявление базовых аугментаций

In [ ]:
tr = Compose([Resize(64, 128, p=1.0),
        Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0)])

## Фиксирование рандомов для воспроизводимости результатов экспериментов

In [ ]:
import random

def set_seed(seed:int=1):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed()

## Объявление функций для подсчета метрик при обучении и валидации 

In [ ]:
from torchmetrics import CharErrorRate


def accuracy(preds:torch.Tensor, label:torch.Tensor):
    result = 0
    for i in range(len(preds)):
        count = 0
        for j in range(len(preds[i])):
            if preds[i][j] == label[i][j]:
                count += 1
            if count == 7:
                result += 1
    return result

def CER(inv_d:dict, lbl:torch.Tensor, preds:torch.Tensor, metric):
    preds = preds.cpu().detach().numpy()
    result_preds, result_lbl = list(), list()
    for i in range(len(preds)):
        tmp_preds, tmp_lbl = '', ''
        for j in range(len(preds[i])):
            tmp_preds += inv_d[preds[i][j]]
            tmp_lbl += lbl[i][j]
        result_preds.append(tmp)
        result_lbl.append(t)
    return metric(result_preds, result_lbl)

## Функция тренировки модели

In [ ]:
def model_train(label_smoothing:float=1e-5, weight_decay:float=1e-5, num_epochs:int=25):
    dataloader_test, data_test_size = CreateDataloader(data_type='test', transforms=tr, shuffle=False, state=True)
    dataloader_train, data_train_size = CreateDataloader(data_type='train', transforms=tr, shuffle=True, state=False) # train
    model = Model()

    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=weight_decay)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5)
    model.to(device)
    metric = CharErrorRate()
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    for ep in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        running = 0
        loss = 0.0
        err = 0
        model.train()
        for img, lbl in tqdm(dataloader_train):
            img = img.to(device, non_blocking=True)
            optimizer.zero_grad()
            out = model(img)
            lbl_convert = ConverLabelToNum(d, lbl).to(device, non_blocking=True)
            _, preds = torch.max(out, 2)
            for out_iter, lbl_iter in zip(out, lbl_convert):
                loss += criterion(out_iter, lbl_iter)
            loss.backward()
            optimizer.step()
            running_loss += loss * out.size(1)
            loss = 0.0
            running_corrects += accuracy(preds, lbl_convert.data)
            running += torch.sum(preds == lbl_convert.data)
            err += CER(inv_d, lbl, preds, metric)
        epoch_loss = running_loss / (data_train_size * out.size(1))
        epoch_acc = running_corrects / (data_train_size * out.size(1))
        print(f'Epoch {ep + 1} train: loss: {epoch_loss},  char_acc: {running / (data_train_size * out.size(1))}, accuracy: {epoch_acc}, CER: {err / data_train_size}')
        running_loss = 0.0
        running_corrects = 0
        running = 0
        loss = 0.0
        err = 0
        model.eval()
        with torch.no_grad():
            for img, lbl in tqdm(dataloader_test):
                img = img.to(device, non_blocking=True)
                optimizer.zero_grad()
                out = model(img)
                lbl_convert = ConverLabelToNum(d, lbl).to(device, non_blocking=True)
                _, preds = torch.max(out, 2)
                for out_iter, lbl_iter in zip(out, lbl_convert):
                    loss += criterion(out_iter, lbl_iter)
                running_loss += loss * out.size(1)
                loss = 0.0
                running_corrects += accuracy(preds, lbl_convert.data)
                running += torch.sum(preds == lbl_convert.data)
                err += CER(inv_d, lbl, preds, metric)
            epoch_loss = running_loss / (data_test_size * out.size(1))
            epoch_acc = running_corrects / (data_test_size * out.size(1))
            print(f'Epoch {ep + 1} val: loss: {epoch_loss}, char_acc: {running / (data_test_size * out.size(1))}, accuracy: {epoch_acc}, CER: {err / data_test_size}')
            scheduler.step(epoch_loss)
    return model

In [ ]:
model_train(label_smoothing=1e-2, weight_decay=1e-3, num_epochs=30)

100%|██████████| 782/782 [07:36<00:00,  1.71it/s]


Epoch 1 train: loss: 1.8567851781845093,  char_acc: 0.518879771232605, accuracy: 0.006472075779006472, CER: 0.0018750018207356334


100%|██████████| 40/40 [00:13<00:00,  2.91it/s]


Epoch 1 val: loss: 0.7625036239624023, char_acc: 0.8160673379898071, accuracy: 0.03701798751303702, CER: 0.0007356948917731643


100%|██████████| 782/782 [07:26<00:00,  1.75it/s]


...

100%|██████████| 782/782 [07:28<00:00,  1.74it/s]


Epoch 30 train: loss: 0.3957415819168091,  char_acc: 0.9103460311889648, accuracy: 0.10028431414570028, CER: 0.00034999812487512827


100%|██████████| 40/40 [00:13<00:00,  2.90it/s]


Epoch 30 val: loss: 0.26697391271591187, char_acc: 0.9521952271461487, accuracy: 0.11078250682211079, CER: 0.00019200582755729556


Model(
  (conv1): Sequential(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Sequential(
    (0): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Sequential(
    (0): Linear(in_features=512, out_features=32, bias=True)
    (1): ReLU(inplace=True)
  )
  (gru1): GRU(32, 512)
  (gru1_b): GRU(32, 512)
  (gru2): GRU(512, 512)
  (gru2_b): GRU(512, 512)
  (fc2): Sequential(
    (0): Linear(in_features=1024, out_features=66, bias=True)
    (1): ReLU(inplace=True)
  )
)

# Подведение итогов

Были выполнены все условия предложенной работы, за исключением анализа ошибок.

Максильные результаты по метрикам не были достигнуты, поскольку была проведена только часть экспериментов. Улчшение качества можно получить потем более точного подбора аугментаций (в работе был применен Autoaugment -- он преобразует изображение только в цветовой палитре). 

Также можно провести эксперименты с масштабированием применяемой архитектуры, посмотреть на результаты: с разными шедулерами, TTA, ema, оптимизаторами (к ним можно запустить поиск гиперпарамтров), L2, докинуть на полносвязные слои Dropout.
